In [1]:
# импортируем необходимые библиотеки, классы и функции

# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс PowerTransformer, позволяющий
# выполнить преобразование Бокса-Кокса/Йео-Джонсона
# и стандартизацию
from sklearn.preprocessing import PowerTransformer
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс KFold и функцию cross_val_score
# для выполнения перекрестной проверки
from sklearn.model_selection import KFold, cross_val_score
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Example.csv', sep=';')

In [3]:
# смотрим первые 5 наблюдений
data.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,"27,09",0,"39,74",Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680,0
1,NaN,0,"46,31",Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,"37111,5",0
2,"23,76",0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,"37079,4",0
3,"9,4",NaN,"13,9",Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997,0
4,"14,15",0,"108,43",Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,"16829,6",0


In [4]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['longdist', 'internat', 'local', 'income']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [5]:
# удаляем лишние символы в категориях переменных
# gender и marital
for i in ['gender', 'marital']:
    data[i] = data[i].str.replace('[*&_]', '')

In [6]:
# заменяем редкую категорию модой
data.at[data['pay'] == 'CD', 'pay'] = 'CC'

In [7]:
# пишем функцию, создающую парные взаимодействия
def make_conj(df, feature1, feature2):
    df[feature1 + "_" + feature2] = df[feature1].astype('object') + " + " + df[feature2].astype('object')

In [8]:
# применяем функцию
make_conj(data, 'gender', 'marital')

In [9]:
# поделим возраст на длительность междугородних звонков в минутах
data['ratio'] = data['age'] / data['longdist']
# заменяем бесконечные значения на 1
data['ratio'].replace([np.inf, -np.inf], 1, inplace=True)

In [10]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
data['ratio2'] = data['longdist'] / data['internat']
# заменяем бесконечные значения на 0
data['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)

In [11]:
# поделим доход на возраст
data['ratio3'] = data['income'] / data['age']
# заменяем бесконечные значения на 0
data['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)

In [12]:
# поделим возраст на количество детей
data['ratio4'] = data['age'] / data['children']
# заменяем бесконечные значения на 0
data['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [13]:
# создаем массив меток
y = data.pop('churn').values

In [14]:
# подготавливаем данные перед преобразованием Бокса-Кокса
# (данные должны быть положительными)
data.replace({0: 0.5}, inplace=True)

In [15]:
# создаем списки количественных и категориальных столбцов
num_columns = data.dtypes[data.dtypes != 'object'].index
cat_columns = data.dtypes[data.dtypes == 'object'].index

In [16]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

In [17]:
# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [18]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]
# передаем список в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [19]:
# задаем 10-блочную перекрестную проверку с перемешиванием
kf = KFold(n_splits=10, shuffle=True, random_state=123)

In [20]:
# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', transformer), 
                    ('logreg', LogisticRegression(solver='lbfgs', 
                                                  max_iter=200))])
# выполняем перекрестную проверку, конвейер размещен
# внутри цикла перекрестной проверки
cross_val_score(ml_pipe, data, y, scoring='roc_auc', cv=kf).mean()

0.8835622189984413